# Plotting Code
The data is read from the data/results/results_csv folder. The data was exportet to csv files from the bag files by using the csv export function in plotjuggler. 

In [ ]:
#define matplotlib backend
%matplotlib

#imports
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
import os
from scipy.spatial.transform import Rotation as R

In [ ]:
#function for removing zeros from list
def remove_zeroes(data):
    data = data[np.logical_not(np.isnan(data))]
    return data

## plot xy results of all localization methods at once for kreuzmuster2

In [ ]:
filename = 'data/results/results_csv/kreuzmuster2_high_proc_cov.csv'
delimiter = ','

#read csv file
csv_dataframe = pd.read_csv(filename, delimiter=delimiter)
header = list(csv_dataframe.columns.values)

#define columns to be extracted from csv dataframe, order: amcl, marv, ekf, faro
selected_columns = [["/amcl_pose/pose/pose/position/x"       ,"/amcl_pose/pose/pose/position/y"         ],
                    ["/transformed_pos/pose/pose/position/x" ,"/transformed_pos/pose/pose/position/y"   ],
                    ["/ekf_pose/pose/pose/position/x"        ,"/ekf_pose/pose/pose/position/y"          ],
                    ["/faro_pos/pose/position/x"             ,"/faro_pos/pose/position/y"               ]]

fig,ax = plt.subplots()
selector = 0
color = iter(['#ff0000','#0000ff','#00ff00','#000000']) #define colors

for selector in range(4):
    c = next(color)

    xvals = csv_dataframe[selected_columns[selector][0]]
    yvals = csv_dataframe[selected_columns[selector][1]]
    
    xvals = remove_zeroes(xvals)
    yvals = remove_zeroes(yvals)
    
    ax.plot(xvals,yvals,'.-',markersize=1,c=c)
    ax.set(xlabel='x (m)', ylabel='y (m)', title='kreuzmuster 2')
    ax.legend(['AMCL','Marvelmind','EKF', 'faro'])
    ax.grid(True)

plt.show()

## plot x and y results of all localization methods seperately for kreuzmuster2

In [ ]:
filename = 'data/results/results_csv/kreuzmuster2_high_proc_cov.csv'
delimiter = ','

csv_dataframe = pd.read_csv(filename, delimiter=delimiter)
header = list(csv_dataframe.columns.values)

#define columns to be extracted from csv dataframe, order: amcl, marv, ekf, faro
selected_columns = [["/amcl_pose/pose/pose/position/x"       ,"/amcl_pose/pose/pose/position/y"         ],
                    ["/transformed_pos/pose/pose/position/x" ,"/transformed_pos/pose/pose/position/y"   ],
                    ["/ekf_pose/pose/pose/position/x"        ,"/ekf_pose/pose/pose/position/y"          ],
                    ["/faro_pos/pose/position/x"             ,"/faro_pos/pose/position/y"               ]]

timestamp = csv_dataframe["__time"]
timestamp = timestamp - timestamp[0] #set start time to 0

fig,ax = plt.subplots()
selector = 0
color = iter(['#ff0000','#0000ff','#00ff00','#000000']) #define colors

# x values
for selector in  range(4):
    c = next(color)

    xvals = csv_dataframe[selected_columns[selector][0]].interpolate()
    yvals = csv_dataframe[selected_columns[selector][1]].interpolate()
    
    ax.plot(timestamp,xvals,'.-',markersize=0.5,c=c)
    ax.set(xlabel='t (s)', ylabel='y (m)', title='kreuzmuster_2 x')
    ax.legend(['AMCL','Marvelmind','EKF', 'faro'])
    ax.grid(True)
plt.show()

fig,ax = plt.subplots()
selector = 0
color = iter(['#ff0000','#0000ff','#00ff00','#000000']) #define colors

#y values
for selector in  range(4):
    c = next(color)

    xvals = csv_dataframe[selected_columns[selector][0]].interpolate()
    yvals = csv_dataframe[selected_columns[selector][1]].interpolate()

    ax.plot(timestamp,yvals,'.-',markersize=0.5,c=c)
    ax.set(xlabel='t (s)', ylabel='y (m)', title='kreuzmuster_2 y')
    ax.legend(['AMCL','Marvelmind','EKF', 'faro'])
    ax.grid(True)
plt.show()

## display angle data for drift visualization using double90 bag

In [ ]:
filename = 'data/results/results_csv/double90_4_odom_drift.csv'
delimiter = ','

csv_dataframe = pd.read_csv(filename, delimiter=delimiter)
header = list(csv_dataframe.columns.values)

#define columns to be extracted from csv dataframe
selected_columns = [["/amcl_pose/pose/pose/orientation/yaw_deg"         ,"/amcl_pose/pose/pose/orientation/x"       ,"/amcl_pose/pose/pose/orientation/y"       ,"/amcl_pose/pose/pose/orientation/z"       ,"/amcl_pose/pose/pose/orientation/w"      ],
                    ["/transformed_pos/pose/pose/orientation/yaw_deg"   ,"/transformed_pos/pose/pose/orientation/x" ,"/transformed_pos/pose/pose/orientation/y" ,"/transformed_pos/pose/pose/orientation/z" ,"/transformed_pos/pose/pose/orientation/w"],
                    ["/odom/pose/pose/orientation/yaw_deg"              ,"/odom/pose/pose/orientation/x"            ,"/odom/pose/pose/orientation/y"            ,"/odom/pose/pose/orientation/z"            ,"/odom/pose/pose/orientation/w"           ]]

fig,ax = plt.subplots()
selector = 0
color = iter(['#ff0000','#0000ff','#00ff00','#000000']) #define colors

offsets = [178,88,102] #define absolute offsets (measurend manually in plotjuggler)

for selector in  range(3):
    c = next(color)
    xaxis = csv_dataframe["__time"]
    xaxis = xaxis - xaxis[0]

    #seperate quaternion data
    yaw =    csv_dataframe[selected_columns[selector][0]]
    quat_x = csv_dataframe[selected_columns[selector][1]]
    quat_y = csv_dataframe[selected_columns[selector][2]]
    quat_z = csv_dataframe[selected_columns[selector][3]]
    quat_w = csv_dataframe[selected_columns[selector][4]]
    
    #convert quaternion to euler
    new_yaw = []
    for i in range(len(yaw)):
        try:
            r = R.from_quat([quat_x[i],quat_y[i],quat_z[i],quat_w[i]])
            new_yaw.append(r.as_euler('xyz', degrees=True)[2])
        except:
            new_yaw.append(np.nan)
    print(sum(new_yaw))

    #add offsets
    for i in range(len(new_yaw)):
        new_yaw[i] = new_yaw[i] + offsets[selector]

    #plot
    ax.plot(xaxis,new_yaw,'.-',markersize=1,c=c)
    ax.set(xlabel='t(s)', ylabel='yaw (°)', title='rotations yaw')
    ax.legend(['AMCL','Marvelmind','Odometrie'])
    ax.grid(True)
plt.show()               

## display angle data for ekf performance visualization using double90 bag

In [ ]:
filename = 'data/results/results_csv/rotations4.csv'
delimiter = ','

csv_dataframe = pd.read_csv(filename, delimiter=delimiter)
header = list(csv_dataframe.columns.values)

#define columns to be extracted from csv dataframe
selected_columns = [["/amcl_pose/pose/pose/orientation/yaw_deg","/amcl_pose/pose/pose/orientation/x","/amcl_pose/pose/pose/orientation/y","/amcl_pose/pose/pose/orientation/z","/amcl_pose/pose/pose/orientation/w"],
                    ["/transformed_pos/pose/pose/orientation/yaw_deg" ,"/transformed_pos/pose/pose/orientation/x","/transformed_pos/pose/pose/orientation/y","/transformed_pos/pose/pose/orientation/z","/transformed_pos/pose/pose/orientation/w"],
                    ["/odometry/filtered/pose/pose/orientation/yaw_deg" ,"/odometry/filtered/pose/pose/orientation/x","/odometry/filtered/pose/pose/orientation/y","/odometry/filtered/pose/pose/orientation/z","/odometry/filtered/pose/pose/orientation/w"]]
fig,ax = plt.subplots()

selector = 0
color = iter(['#ff0000','#0000ff','#00ff00','#000000'])

for selector in  range(3):
    c = next(color)
    xaxis = csv_dataframe["__time"]
    xaxis = xaxis - xaxis[0]

    #seperate quaternion data
    yaw =    csv_dataframe[selected_columns[selector][0]]
    quat_x = csv_dataframe[selected_columns[selector][1]]
    quat_y = csv_dataframe[selected_columns[selector][2]]
    quat_z = csv_dataframe[selected_columns[selector][3]]
    quat_w = csv_dataframe[selected_columns[selector][4]]
    
    #convert quaternion to euler
    new_yaw = []
    for i in range(len(yaw)):
        try:
            r = R.from_quat([quat_x[i],quat_y[i],quat_z[i],quat_w[i]])
            new_yaw.append(r.as_euler('xyz', degrees=True)[2])
        except:
            new_yaw.append(np.nan)

    #plot
    print(sum(new_yaw))
    ax.plot(xaxis,new_yaw,'.-',markersize=1,c=c)
    ax.set(xlabel='t(s)', ylabel='yaw (°)', title='rotations yaw')
    ax.legend(['AMCL','Marvelmind','EKF', 'odometry_filtered'])
    ax.grid(True)

plt.show()  